# SOSE surface salt sources $F = F_{A \rightarrow O} + F_{I \rightarrow O}$
`SFLUX = E - P - R`

From Ryan's Nature Geoscience 2016 paper, [supplemental doc](https://static-content.springer.com/esm/art%3A10.1038%2Fngeo2749/MediaObjects/41561_2016_BFngeo2749_MOESM303_ESM.pdf) (eqs S5-S9)

In [1]:
from dask_gateway import Gateway
g = Gateway()
cluster = g.connect(g.list_clusters()[0].name)
client = cluster.get_client()

In [2]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.232f8b6ff3b846bdae62df5fe0318db5/status,


In [63]:
client.close()
cluster.close()

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()


In [3]:
import xarray as xr
import numpy as np
import dask.array as dsa
import matplotlib.pyplot as plt
from fastjmd95 import jmd95numba
import json
import fsspec
import gcsfs
import xmitgcm

In [4]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean.yaml")
ds  = cat["SOSE"].to_dask()

In [5]:
coordsr = ds.coords.to_dataset().reset_coords()
dsr = ds.reset_coords(drop=True)
dsr

,Array,Chunk
Bytes,47.37 GiB,110.74 MiB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,439 Tasks,438 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,47.37 GiB,110.74 MiB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,439 Tasks,438 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [22]:
#WG boundaries (65˚W, 30˚E; 80˚S, 62˚S)
#WG model's boundaries (-77.87497˚S, -62.041634˚S; 295.08334W, 30.083334E)
lower_lat = 0
upper_lat = 95
left_lon =  310
right_lon = 880

#set the boundaries
lat_range = dict(YC=slice(lower_lat, upper_lat))
lon_range = dict(XC=slice(left_lon, right_lon))

coords = coordsr.roll(XC=700, roll_coords=True).isel(**lat_range, **lon_range)
ds = dsr.roll(XC=700, roll_coords=True).isel(**lat_range, **lon_range)

sit = ds.SIheff

In [23]:
sit

<xarray.DataArray 'SIheff' (time: 438, YC: 95, XC: 570)>
dask.array<getitem, shape=(438, 95, 570), dtype=float32, chunksize=(1, 95, 570), chunktype=numpy.ndarray>
Coordinates:
  * XC       (XC) float32 295.1 295.2 295.4 295.6 ... 29.42 29.58 29.75 29.92
  * YC       (YC) float32 -77.87 -77.71 -77.54 -77.37 ... -62.54 -62.37 -62.21
  * time     (time) datetime64[ns] 2005-01-06 2005-01-11 ... 2010-12-31
Attributes:
    long_name:      SEAICE effective ice thickness
    standard_name:  SIheff
    units:          m

In [30]:
ice_atm2ocean = ds.SIempmr
atm2ice_atm2ocean = ds.SIatmFW
atm2ice = ds.SIsnPrcp

lhs = ds.SFLUX
rhs = ice_atm2ocean - atm2ice_atm2ocean + atm2ice
#rhs = ds.SIempmr - ds.SIatmFW + ds.SIsnPrcp

In [34]:
diff = lhs - rhs

In [35]:
diff.std().drop('XC').plot()

KeyboardInterrupt: 